## 1 PID Concept

## 1.1 Exercise 1
Here are some questions about the PID-controller. Use your own words.

> (a) Why a pure P-controller will generate stationary error/offset when used for
most of the existing systems?

A P-controller applies output proportional to the error. For many systems this will reach a equlibrium *near* the setpoint, but never reaching it. As Kp gain is higher one can get closer, but at risk of overshoot or instability.

> (b) Why the addition of the integral term may remove the stationary error in most
of the cases?

A stationary error will, if not removed, cause the integral of the error to grow over time.
With an I-controller high integral values causes it to take action to counteract, which elimitates the stationary error. 

> (c) Which controller is the fastest and what controller is the slowest among the
pure P-controller, I-controller and D-controller? What does this mean for
systems with high-frequency motion and systems with low frequency motion?

P is faster than I, as I requires values to build up over time. D is theoretically faster than P, as its value suggests potential future values of the system. 

When a fast system is needed and a bit of stationary error is acceptable and measurement/process noise is low, a PD controller can be used. If a system does is OK with slow response time, one can use a PI regulator. Not using a D-term can make the system more stable in presence of noise.

> (d) What is the main motivation for introducing the D-gain in the controller? Can
you suggest another way to achieve the same effect you described?

The D-gain is used to counteract oscillations around the setpoint. As a system with inertia or time-delay has been set into motion towards a setpoint, the amount of should be reduced before one gets to the setpoint - or overshoot will happen. The P term does not do this fast enough.

> (e) Sum up the main effects of each term P, I and D individually on: (i) response
time; (ii) stationary error/offset; (iii) overshoot; (iv) noise; (v) stability

```
                 P            I             D

Response time.  Decreases.   Decreases.    Faster settle time 
Error offset.   Decreases.   Eliminates.   No effect
Overshoot.      Increases.   Increases.    Decreases
Noise.          Minor.       Minor.        Increases
Stability.      Decreases.   Decreases.    Sometimes improves 
```

## 1.2 Exercise 2

In MATLAB/Simulink, create a PID controller for the system plant
$G(s) = \frac{Y(s)}{X(s)} = \frac{s+1}{s^2−5s+6}$
to track its output y(t) to x(t) = 1. Find a suitable tuning for that.

Plot cases for pure P-controller, PI-controller, and PID-controller.
Discuss about the differences.

#### P controller
with Kp=30
![P controller response](./ass2-ex2-P30.png)

Stabilizes significantly below the setpoint.

#### PI controller
with Kp=50, Ki=50 
![PI controller response](./ass2-ex2-P50-I50.png)

Now correctly holds the setpoint. Overshoot is a bit lower also. Rise time is practically identical.

#### PID controller
with Kp=50, Ki=50, Kd=0.005
![PID controller response](./ass2-ex2-P50-I50-D0005.png)

Almost identical to PI controller.

In [1]:
%matplotlib inline
import numpy
import scipy
import scipy.signal
import matplotlib.pyplot as plt
import pandas

Transfer function of PID regulator $$\frac{Kd s^2 + Kp s + Ki}{s}$$

# 2 Implementation on AUV
Now, let us use the controller for the AUV discussed in the List 1.

## 2.1

> (a) Use the dynamic equations that we found in Exercise 1. Assume that our goal
is to make the AUV move with constant velocity v_d . Find the expression for
a control signal u(t) by using a P-controller.

$$
\begin{align}
\dot{v} & = \frac{u}{m} - \frac{k}{m}v \\
\dot{v} & = -\frac{k}{m}v + \frac{1}{m}u \\
\dot{x} & = ax + bu && \text{with a=-k/m and b=1/m}
\end{align}
$$

With a P controller we set `u` based on the error of the system. The error is `v - r`, where `r` is the desired speed (setpoint), and we use a factor `kP` to adjust how strongly we apply the regulation.

$$
\begin{align}
\dot{v} & = -\frac{k}{m}v + \frac{1}{m}u \\
\dot{v} & = -\frac{k}{m}v + \frac{1}{m} * kP(v - r) \\
\end{align}
$$

TODO

> Find the dynamic equations in closed loop.

TODO

> (b) Find the new gain, stationary value and transient in closed loop with the P-
controller as a function of Kp.

TODO

> What is the effect of increasing Kp? Point out
the positive and negative effects of augmenting the gain Kp.

TODO

> (c) Find the dynamics of the AUV in closed loop with a PID-controller. Write
the system as a first order differential equation in matrix form.

TODO

> (d) Draw the block diagram for the system with PID controller.

TODO

> (e) Assume, as in Exercise 1, that m = 200kg and k = 100kg/s . Implement the
block diagram in MATLAB/Simulink. Find values for Kp, Ki, Kd that give
us fast response with little oscillations. Remember to choose feasible/realistic
values in the real world, for example Kp < 5 and not Kp > 10000000 (which
would demand a gigantic energy cost).

TODO: simulate

> (f) Set Kd = 0 . Tune Kp and Ki so that the system becomes critically damped

TODO: simulate

> (g) Find the dynamic equations for the system with the PI-controller.
Based on it, calculate values for Kp and Ki so that the system becomes critically damped .
Do the values correspond with the values that you found by tuning in the
previous item (f)?

# 3. Digital PID

## 3.1 Exercise 4
> (a) What is the difference between the Backward Euler, Forward Euler and Tustin
method for implementing the integral or derivative part?

Forward Euler uses explicit formulation. Easy and fast implementation. Introduces an error at each step. If timestep is too large, will be numerically unstable.

Backward Euler. Implicit formulation. Numerically stable (converges) even for very large timesteps.

Tustin method. Uses the bilinear transform approximation to go from s to z-domain. Can be is better at approximating the frequency response of the continious system than Forward or Backward Euler methods.

> (b) Write an equation for the PID-controller in digital domain, where the Back-
ward Euler method is used. Use the time period T.


In continious-time domain we can express the PID controller as: $\dot{u}(t) = K_p \dot{e}(t) + K_p T_i e(t) + K_p T_d \ddot{e}(t)$

where `e(t)` is the difference between setpoint `r` and process measurement `y`: $e(t) = r(t) − y(t)$

Backward Euler differentiation: $\dot{x}(t_k) ≈ \frac{x(t_k) − x(t_{k−1})}{T}$

By applying first to $\dot{u}, \dot{e}, \ddot{e}$ and then again to the $\dot{e(t_k)},\dot{e(t_{k-1})}$ and solving for $u(t_k)$ one can find:

$$
\begin{align}
u(t_k) & = u(t_{k−1}) \\
      & + K_p [e(t_k) − e(t_{k−1})] \\
      & + \frac{K_p T}{T_i} e(t_k) \\
      & + \frac{K_p T_d}{T} [e(t_k)−2e(t_{k−1})+e(t_{k−2})] \\
\end{align}
$$


> (c) What is the effect of a high time period in the system stability?

A high time period may destabilize the system because of the time delay introduced by the sampling makes the measurement signal too discontinious. As a rule of thumb the sampling period should be less than 1/10 of the response time of the system (time to reach 63% of setpoint after a step change).



> (d) Draw a block diagram for the digital PID controller. Include the block of A/D
and D/A converter.

![Discrete PID using Backward Euler](./assignment2-pid-discrete2.png)